In [1]:
from test_case_agent import testcase_agent
from optimization_agent import optimization_suggestor_agent

## Input

In [2]:
code = """def calculate_factorials_up_to_number(n):
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1]  # Initialize with the factorial of 0

    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
        factorials.append(factorial)

    return factorials"""

## Initialize Testcase Agent

In [3]:
agent = testcase_agent(code = code,
                    #    test_case_list =["calculate_factorials_up_to_number(100000)"]
                       )

In [4]:
vas = agent.run_flow()

-----test case generation---
Worst case
```python
calculate_factorials_up_to_number(1000)
```

Medium case
```python
calculate_factorials_up_to_number(10)
```

Best case
```python
calculate_factorials_up_to_number(0)
```


-----test case extraction---
calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------
None


-----test case merger---
import time
import sys
import os
import psutil

def memory_usage_psutil():
    # return the memory usage in MB
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss

def test_case():
    start_time = time.time()
    start_memory = memory_usage_psutil()

    # Call the function here
    calculate_factorials_up_to_number(1000)
    
    end_time = time.time()
    end_memory = memory_usage_psutil()

    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Memory used: {end_memory - start_memory} B")

def calcul

In [5]:
from pprint import pprint 
pprint(vas)

{'response': [{'Memory used': 524288.0, 'Time taken': 0.0005857944488525391},
              {'Memory used': 0.0, 'Time taken': 0.00013208389282226562},
              {'Memory used': 0.0, 'Time taken': 0.0001239776611328125}],
 'testcase': ['calculate_factorials_up_to_number(1000)',
              'calculate_factorials_up_to_number(10)',
              'calculate_factorials_up_to_number(0)']}


------

## Optimize Code

In [8]:
op_agent = optimization_suggestor_agent(code= code,
                                        execution_obj= vas)

In [9]:
vas = op_agent.run_flow()

----raw optimizations----
The code you provided calculates and returns a list of factorials up to a given number 'n'. Here are some suggestions for optimizing the code:

1. Error Handling: The code checks if the input 'n' is a non-negative integer and raises a `ValueError` if it's not. It would be better to use a built-in function, such as `isinstance(n, int)`, to check if 'n' is an integer and `n >= 0` to check if it's non-negative. This would simplify the error handling and remove the need to raise an exception explicitly.

2. Use List Comprehension: Instead of initializing an empty list `factorials` and then appending to it in each iteration, you can use list comprehension to generate the list directly. This would eliminate the need for the explicit loop and the `append()` method.

3. Calculate the Factorial Incrementally: The code currently multiplies the factorial with 'i' in each iteration. Instead, you can calculate the next factorial by multiplying the previous factorial with '

In [10]:
print(vas)

def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1] * (n + 1)

    for i in range(2, n + 1):
        factorials[i] = factorials[i - 1] * i

    return factorials


------------------

In [23]:
extracted_blocks = """def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = {}

    def factorial(k):
        if k in factorials:
            return factorials[k]
        elif k == 0:
            return 1
        else:
            result = k * factorial(k - 1)
            factorials[k] = result
            return result

    return [factorial(i) for i in range(n + 1)]"""

## Optimized Code Testcases

In [24]:
op_code_test = testcase_agent(
    code = extracted_blocks)

In [25]:
op_code_test.gen_test_case()
op_code_test.extract_test_cases()
op_code_test.join_test_case()
op_code_test.run_test_case()
vas = op_code_test.extract_info_tg()

calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------


In [26]:
print(vas)

[{'Time taken': 0.0008258819580078125, 'Memory used': 655360.0}, {'Time taken': 0.00017380714416503906, 'Memory used': 0.0}, {'Time taken': 0.00018930435180664062, 'Memory used': 0.0}]


----

## Profiler

In [1]:
from profiler_agent import profiler

In [2]:
code = """def do_stuff(numbers):
    s = sum(numbers)
    prod = 1
    for n in numbers:
        prod *= n
    return s, prod"""
testcase = "do_stuff([1, 2, 3, 4, 50000000000000000000000])"

In [3]:
p_agent = profiler(
    code= code,
    testcase= testcase
                   )

In [4]:
vas = p_agent.run_flow()

command ran
dict_keys(['s = sum(numbers)', 'prod = 1', 'for n in numbers:', 'prod *= n', 'return s, prod'])
def do_stuff(numbers):
    s = sum(numbers) #percent_time: 32.2%
    prod = 1 #percent_time: 5.0%
    for n in numbers: #percent_time: 28.8%
        prod *= n #percent_time: 28.8%
    return s, prod #percent_time: 5.2%


In [5]:
print(vas)

def do_stuff(numbers):
    s = sum(numbers) #percent_time: 32.2%
    prod = 1 #percent_time: 5.0%
    for n in numbers: #percent_time: 28.8%
        prod *= n #percent_time: 28.8%
    return s, prod #percent_time: 5.2%


---------

In [1]:
from overseer import flow_master

In [2]:
code = """def calculate_factorials_up_to_number(n):
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1]  # Initialize with the factorial of 0

    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
        factorials.append(factorial)

    return factorials"""

In [3]:
flow = flow_master(code= code)

In [5]:
flow.start()

-----test case generation---
Worst case
```python
calculate_factorials_up_to_number(1000)
```

Medium case
```python
calculate_factorials_up_to_number(10)
```

Best case
```python
calculate_factorials_up_to_number(0)
```


-----test case extraction---
calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------
None


-----test case merger---
import time
import sys
import os
import psutil

def memory_usage_psutil():
    # return the memory usage in MB
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss

def test_case():
    start_time = time.time()
    start_memory = memory_usage_psutil()

    # Call the function here
    calculate_factorials_up_to_number(1000)
    
    end_time = time.time()
    end_memory = memory_usage_psutil()

    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Memory used: {end_memory - start_memory} B")

def calcul

In [1]:
code = """import math

def calculate_factorials_up_to_number(n):
    assert n >= 0, "Input must be a non-negative integer."

    factorials = [math.factorial(i) for i in range(n + 1)]

    return factorials"""

testcase = ["calculate_factorials_up_to_number(1000)", "calculate_factorials_up_to_number(100)", "calculate_factorials_up_to_number(10)"]

In [2]:
from executor_agent import code_executor

In [3]:
ce = code_executor(
    code = code,
    testcase = testcase
)

In [4]:
ce.run()

[{'Time taken': 0.012521505355834961, 'Memory used': 655360.0},
 {'Time taken': 0.0001900196075439453, 'Memory used': 0.0},
 {'Time taken': 0.00023603439331054688, 'Memory used': 0.0}]